In [9]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
from itertools import product

import os, sys
sys.path.append(os.path.join(os.getcwd(), '../sympdee/sympde'))
from misc.utils import write_lines
from data.pdes import PDEs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
def generate_combinations(dictionary):
    # Base case: if the dictionary is empty, return an empty list
    if not dictionary:
        return [""]

    # Take one key-value pair from the dictionary
    key, values = next(iter(dictionary.items()))
    remaining_dict = dictionary.copy()
    del remaining_dict[key]

    # Recursively generate combinations for the remaining dictionary
    combinations = generate_combinations(remaining_dict)

    # Generate all possible combinations with the current key-value pair
    output_lines = []
    for value in values:
        for combination in combinations:
            if value is None:
                line = combination
            elif combination == "":
                line = f"--{key} {value}" 
            else:
                if isinstance(value, list):
                    value = " ".join([str(v) for v in value])
                line = f"{combination} --{key} {value}"
            output_lines.append(line)

    return output_lines

def dict_to_array(comb_dict):
    output_lines = generate_combinations(comb_dict)
    output_lines = "\n".join(output_lines)
    return output_lines

In [11]:
eps0to1 = np.linspace(0, 1, 3)
eps_0to1s = [[0] + [np.round(eps,1) for eps in epss] for epss in product(eps0to1, eps0to1, eps0to1)]

In [12]:
eps0to1s_nogrid = [[0,0,0,0]]
eps1 = np.linspace(0, 1, 5)[1:]
eps0 = [0]*4
for i in range(1,4):
    for eps1_i in range(len(eps1)):
        eps0_i = eps0.copy()
        eps0_i[i] = eps1[eps1_i]
        eps0to1s_nogrid.append(eps0_i)

eps0to1s_nogrid

[[0, 0, 0, 0],
 [0, 0.25, 0, 0],
 [0, 0.5, 0, 0],
 [0, 0.75, 0, 0],
 [0, 1.0, 0, 0],
 [0, 0, 0.25, 0],
 [0, 0, 0.5, 0],
 [0, 0, 0.75, 0],
 [0, 0, 1.0, 0],
 [0, 0, 0, 0.25],
 [0, 0, 0, 0.5],
 [0, 0, 0, 0.75],
 [0, 0, 0, 1.0]]

In [13]:
pdes = PDEs()
pde_names = [pde for pde in pdes]
pde_names

['Pde1',
 'Pde2',
 'Pde3',
 'Pde4',
 'Pde5',
 'Pde6',
 'Pde7',
 'Pde8',
 'Pde9',
 'Pde10',
 'Pde11',
 'Pde12',
 'Pde13',
 'Pde14',
 'Pde15',
 'Pde16',
 'Pde17',
 'Pde18',
 'Pde19',
 'Pde20',
 'Pde21',
 'Pde22',
 'Pde23',
 'Pde24',
 'KdV']

In [14]:
mlp_hidden_channels = [[100]*n_channels for n_channels in range(1, 10)]
mlp_hidden_channels

[[100],
 [100, 100],
 [100, 100, 100],
 [100, 100, 100, 100],
 [100, 100, 100, 100, 100],
 [100, 100, 100, 100, 100, 100],
 [100, 100, 100, 100, 100, 100, 100],
 [100, 100, 100, 100, 100, 100, 100, 100],
 [100, 100, 100, 100, 100, 100, 100, 100, 100]]

In [15]:
mlp_hidden_channels_flat = [[2**16, 2**13, 2**8]]

In [16]:
seeds = [1, 2, 3, 4, 5]

In [17]:
n_splits_train = [1_000, 10_000, 100_000]
n_splits_valtest = 5_000
n_splits = [[n_split_train, n_splits_valtest, n_splits_valtest] for n_split_train in n_splits_train]

In [20]:
lrs = [0.1, 0.01, 0.001, 0.0001]
net = ['TrainP']

In [23]:

comb_dicts = {
    # 'aug_noaug' : {
    #     'epsilons' : [None, [0, 1., 0.1, 0.4]],
    #     'seed': [1, 2, 3, 4, 5],
    # },
    # 'train' : {
    # },
    # 'eps_0to1_nogrid' : {
    #     'epsilons' : eps0to1s_nogrid,
    #     'seed':  seeds,
    # },
    # 'lconv' : {
    #     'rot_angle' : [0, 10, 45, 90],
    #     'dim':  [4, 9, 16, 25],
    #     'digit' : [None, 0, 5],
    # },
    # 'pdes' : {
    #     'pde_name' : pde_names,
    #     'seed':  seeds,
    # },
    # 'mlp_hidden_channels' : {
    #     'mlp_hidden_channels' : mlp_hidden_channels,
    #     'pde_name' : ['Pde22', 'Pde24', 'KdV'],
    #     'seed':  seeds,
    # },
    # 'equiv_mag_none' : {
    #     'equiv' : ['mag', 'none'],
    #     'pde_name' : pde_names,
    #     'seed':  seeds,
    # }
    # 'mlp_hidden_channels_flat' : {
    #     'mlp_hidden_channels' : mlp_hidden_channels_flat,
    #     'pde_name' : pde_names,
    #     'seed':  seeds,
    # }
    # 'transform_types' : {
    #     'transform_type' :  ['space_translation', 'none'],
    #     'seed':  np.arange(1, 101),
    # }
    'sine2d' : {
        'seed':  seeds,
        'n_splits' :  n_splits,
        'lr' : lrs,
        'net' : net,
    }
}


array_dir = '../jobs/arrays/'
for file_name, comb_dict in comb_dicts.items():
    output_file = os.path.join(array_dir, file_name + '.txt')
    output_lines = dict_to_array(comb_dict)
    print(f'### {file_name} ###', comb_dict, output_lines, '\n\n',sep='\n')
    write_lines(output_file, output_lines)


### sine2d ###
{'seed': [1, 2, 3, 4, 5], 'n_splits': [[1000, 5000, 5000], [10000, 5000, 5000], [100000, 5000, 5000]], 'lr': [0.1, 0.01, 0.001, 0.0001], 'net': ['TrainP']}
--net TrainP --lr 0.1 --n_splits 1000 5000 5000 --seed 1
--net TrainP --lr 0.01 --n_splits 1000 5000 5000 --seed 1
--net TrainP --lr 0.001 --n_splits 1000 5000 5000 --seed 1
--net TrainP --lr 0.0001 --n_splits 1000 5000 5000 --seed 1
--net TrainP --lr 0.1 --n_splits 10000 5000 5000 --seed 1
--net TrainP --lr 0.01 --n_splits 10000 5000 5000 --seed 1
--net TrainP --lr 0.001 --n_splits 10000 5000 5000 --seed 1
--net TrainP --lr 0.0001 --n_splits 10000 5000 5000 --seed 1
--net TrainP --lr 0.1 --n_splits 100000 5000 5000 --seed 1
--net TrainP --lr 0.01 --n_splits 100000 5000 5000 --seed 1
--net TrainP --lr 0.001 --n_splits 100000 5000 5000 --seed 1
--net TrainP --lr 0.0001 --n_splits 100000 5000 5000 --seed 1
--net TrainP --lr 0.1 --n_splits 1000 5000 5000 --seed 2
--net TrainP --lr 0.01 --n_splits 1000 5000 5000 --seed 2
